In [1]:
import pandas as pd

from ddeserts.annotate import add_dvap_columns, add_has_charter_column
from ddeserts.load import load_cvap_data

In [2]:
df = load_cvap_data('State')


In [3]:
# add extra columns
add_dvap_columns(df)

geoname      geoid   tot_est  tot_moe   adu_est  \
table line                                                                 
State 1                  Alabama  04000US01   4876250        0   3779875   
      14                  Alaska  04000US02    737070        0    552675   
      27                 Arizona  04000US04   7050300        0   5414955   
      40                Arkansas  04000US05   2999370        0   2295100   
      53              California  04000US06  39283495        0  30261350   
      66                Colorado  04000US08   5610350        0   4349345   
      79             Connecticut  04000US09   3575075        0   2831240   
      92                Delaware  04000US10    957250        0    753565   
      105   District of Columbia  04000US11    692685        0    568755   
      118                Florida  04000US12  20901635        0  16719175   
      131                Georgia  04000US13  10403845        0   7898605   
      144                 Hawaii  04000US15   1422095        0   1117455   
      157                  Idaho  04000US16   1717750        0   1276605   
      170               Illinois  04000US17  12770630        0   9879105   
      183                Indiana  04000US18   6665705        0   5093210   
      196                   Iowa  04000US19   3139510        0   2410770   
      209                 Kansas  04000US20   2910650        0   2199580   
      222               Kentucky  04000US21   4449050        0   3439745   
      235              Louisiana  04000US22   4664360        0   3561010   
      248                  Maine  04000US23   1335490        0   1082995   
      261               Maryland  04000US24   6018850        0   4677165   
      274          Massachusetts  04000US25   6850555        0   5479295   
      287               Michigan  04000US26   9965265        0   7787385   
      300              Minnesota  04000US27   5563380        0   4267530   
      313            Mississippi  04000US28   2984420        0   2270925   
      326               Missouri  04000US29   6104910        0   4723300   
      339                Montana  04000US30   1050650        0    822265   
      352               Nebraska  04000US31   1914570        0   1440465   
      365                 Nevada  04000US32   2972380        0   2291340   
      378          New Hampshire  04000US33   1348125        0   1088160   
      391             New Jersey  04000US34   8878505        0   6916980   
      404             New Mexico  04000US35   2092455        0   1603980   
      417               New York  04000US36  19572320        0  15463820   
      430         North Carolina  04000US37  10264875        0   7968260   
      443           North Dakota  04000US38    756715        0    580975   
      456                   Ohio  04000US39  11655395        0   9050385   
      469               Oklahoma  04000US40   3932870        0   2975905   
      482                 Oregon  04000US41   4129805        0   3261860   
      495           Pennsylvania  04000US42  12791530        0  10129140   
      508           Rhode Island  04000US44   1057230        0    849885   
      521         South Carolina  04000US45   5020805        0   3918305   
      534           South Dakota  04000US46    870640        0    656530   
      547              Tennessee  04000US47   6709355        0   5204435   
      560                  Texas  04000US48  28260855        0  20922410   
      573                   Utah  04000US49   3096850        0   2173265   
      586                Vermont  04000US50    624315        0    507660   
      599               Virginia  04000US51   8454465        0   6588765   
      612             Washington  04000US53   7404105        0   5760560   
      625          West Virginia  04000US54   1817305        0   1447865   
      638              Wisconsin  04000US55   5790715        0   4509815   
      651                Wyoming  04000US56    581025        0    444655 

In [4]:
df.to_csv('output/states.csv')